1. ID3决策树：
    1. 算法: 从根节点开始，每次选取使信息增益$I(X,Y)$最大的特征进行分类，并对产生的子节点递归进行特征选取和分裂，知道所有节点上的数据都属于同一类为止。
    2. 缺点：如果特征本身较为复杂(例如编号，有序等)，则可能导致过拟合。并且ID3算法直到所有特征都被使用完毕才会停止分裂，这可能导致树的深度过大，影响模型的泛化能力。
2. C4.5决策树：改用信息增益率$R(X,Y)$来选择特征进行分裂，避免了ID3算法的过拟合问题。
    1. 算法: 从根节点开始，每次选取使信息增益率$R(X,Y)$最大的特征进行分类，并对产生的子节点递归进行特征选取和分裂，直到所有节点上的数据都属于同一类为止。
    2. 优点：通过使用信息增益率来选择特征，C4.5算法能够更好地处理连续特征和缺失值，并且可以生成更小的决策树。
3. 代价函数：$C(T)=\sum\limits_{t=1}^{|T|}N_tH_t(T)+\lambda|T|$, 其中$H_t(T)=-\sum\limits_{k}\frac{N_{tk}}{N_t}log\frac{N_{tk}}{N_t}$, $N_t$为节点$t$上的样本数，$N_{tk}$为节点$t$上第$k$类样本的数目，$\lambda$为正则化参数。
    1. 代价函数的第一项是所有节点的熵，第二项是树的复杂度。
    2. 通过最小化代价函数，可以得到最优的决策树结构。
4. CART决策树：CART（Classification and Regression Trees）算法由特征选择，树的生成及剪枝组成，既可以分类也可以回归，统称为决策树
    1. 分类树：
        0. 基尼指数：$Gini(P)=\sum\limits_{k=1}^{K}p_k(1-p_k)$, 其中$p_k$为节点$t$上第$k$类样本的比例。基尼指数越大，样本集合的不确定性越大。
        1. 树的生成：
            1. 设节点的训练数据集为D，计算现有特征对该数据集的基尼指数，对每一个特征A，对其可能取的每个值a，根据样本点对A=a的取值与否，划分为D_1,D_2两部分，根据$Gini(D,A)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2)$计算**在特征A是否取a的条件下，集合D的基尼指数**
            2. 选择基尼指数最小的**特征A和对应切分点a作为最优特征和最优切分点**，将D划分为D_1,D_2两部分
            3. 对D_1,D_2两部分递归进行特征选择和划分，直到满足**停止条件**
            4. 返回CART决策树
        2.  树的剪枝：每次选择最小的alpha也就是减去最小的枝


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 读取数据
data=pd.read_csv("data/train.csv")
print(data.info())
print(data[:5])

# 删去编号、姓名、船票编号3列, inplace表示在原数据上修改
data.drop(columns=['PassengerId','Name','Ticket'],inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                      

In [4]:
feat_ranges={}
cont_feat=['Age', 'Fare'] # 连续特征
bins=10 # 连续特征分箱数,即分成多少个区间

for feat in cont_feat:
    # 数据集中存在缺省值nan，需要用np.nanmin()和np.nanmax()来计算
    min_val=np.nanmin(data[feat])# np.nanmin()计算忽略nan的最小值
    max_val=np.nanmax(data[feat])# np.nanmax()计算忽略nan的最大值
    feat_ranges[feat]=np.linspace(min_val, max_val, bins).tolist() # 生成分箱边界
    print(feat,":") # 查看分类点
    for spt in feat_ranges[feat]:
        print(f'{spt:.4f}')

Age :
0.4200
9.2622
18.1044
26.9467
35.7889
44.6311
53.4733
62.3156
71.1578
80.0000
Fare :
0.0000
56.9255
113.8509
170.7764
227.7019
284.6273
341.5528
398.4783
455.4037
512.3292


In [ ]:
# 只有有限取值的离散特征
cat_feat=['Sex','Pclass','SibSp','Parch','Cabin','Embarked']
for feat in cat_feat:
    data[feat]=data[feat].astype('category') # 转换为类别型数据
    print(f'{feat}:{data[feat].cat.categories}') # 查看类别型数据的类别,.cat表示类别型数据的属性，.categories表示类别
    data[feat]=data[feat].cat.codes.to_list() # 将类别按顺序转换为整数
    print(f'{feat}编码后：', data[feat].unique()) # 查看编码后的唯一值
    ranges=list(set(data[feat]))
    ranges.sort()
    feat_ranges[feat]=ranges # 存储离散特征的取值范围

Sex:Index(['female', 'male'], dtype='object')
Sex编码后： [1 0]
Pclass:Index([1, 2, 3], dtype='int64')
Pclass编码后： [2 0 1]
SibSp:Index([0, 1, 2, 3, 4, 5, 8], dtype='int64')
SibSp编码后： [1 0 3 4 2 5 6]
Parch:Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')
Parch编码后： [0 1 2 5 3 4 6]
Cabin:Index(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31', 'A32',
       ...
       'E8', 'F E69', 'F G63', 'F G73', 'F2', 'F33', 'F38', 'F4', 'G6', 'T'],
      dtype='object', length=147)
Cabin编码后： [ -1  81  55 129 145  49 111  13  63  41 101  23  71  21  80 142 140 122
  12  91  98  52  36 116 138 107  45 141  61 123  18  14  69 144   9  28
  43   8 103  93  87  78 102  83  40 134  46  57  89  54 113   3  31  90
  62  51  74 125  72  35  76 124  65  17  56  85 127 146  59 104  24 131
  79  47 115 128  10  50  53  86 126  97 117 133   1  25  64  96  42 121
 106  39  88  26  27  20  82  77   2  48  75   0 135  29   4  95 110 114
   5  33   7 108 132  58  38  34 109  32  19 139  73 120  84  66 137  15
 10

In [6]:
# 将所有缺省值替换为-1
data.fillna(-1, inplace=True)
for feat in feat_ranges.keys():
    feat_ranges[feat]=[-1] + feat_ranges[feat] # 在每个特征的范围前添加-1，表示缺省值

In [7]:
# 划分训练集与测试集
np.random.seed(0)
feat_names = data.columns[1:]
label_name = data.columns[0]
# 重排下标之后，按新的下标索引数据
data = data.reindex(np.random.permutation(data.index))
ratio = 0.8
split = int(ratio * len(data))
train_x = data[:split].drop(columns=['Survived']).to_numpy()
train_y = data['Survived'][:split].to_numpy()
test_x = data[split:].drop(columns=['Survived']).to_numpy()
test_y = data['Survived'][split:].to_numpy()
print('训练集大小：', len(train_x))
print('测试集大小：', len(test_x))
print('特征数：', train_x.shape[1])

训练集大小： 712
测试集大小： 179
特征数： 8


C4.5的实现

In [8]:
class Node:

    def __init__(self):
        # 内部节点的feat表示用来分类的特征编号，其数字与数据中的顺序对应
        # 叶节点的feat表示该节点对应的分类结果
        self.feat=None
        # 分类值列表，表示按照其中的值向子节点分类
        self.split=None
        # 子节点列表，表示每个分类值对应的子节点
        self.child=[]

class DecisionTree:

    def __init__(self, X, Y, feat_ranges, lbd):
        self.root = Node()
        self.X = X
        self.Y = Y
        self.feat_ranges = feat_ranges # 特征取值范围
        self.lbd = lbd # 正则化系数
        self.eps = 1e-8 # 防止数学错误log(0)和除以0
        self.T = 0 # 记录叶节点个数
        self.ID3(self.root, self.X, self.Y)

    # 工具函数，计算 a * log a
    def aloga(self, a):
        return a * np.log2(a + self.eps)

    # 计算某个子数据集的熵
    def entropy(self, Y):
        cnt = np.unique(Y, return_counts=True)[1] # 统计每个类别出现的次数
        N = len(Y)
        ent = -np.sum([self.aloga(Ni / N) for Ni in cnt])
        return ent

    # 计算用feat <= val划分数据集的信息增益
    def info_gain(self, X, Y, feat, val):
        # 划分前的熵
        N = len(Y)
        if N == 0:
            return 0
        HX = self.entropy(Y)
        HXY = 0 # H(X|Y)
        # 分别计算H(X|X_F<=val)和H(X|X_F>val)
        Y_l = Y[X[:, feat] <= val]
        HXY += len(Y_l) / len(Y) * self.entropy(Y_l)
        Y_r = Y[X[:, feat] > val]
        HXY += len(Y_r) / len(Y) * self.entropy(Y_r)
        return HX - HXY

    # 计算特征feat <= val本身的复杂度H_Y(X)
    def entropy_YX(self, X, Y, feat, val):
        HYX = 0
        N = len(Y)
        if N == 0:
            return 0
        Y_l = Y[X[:, feat] <= val]
        HYX += -self.aloga(len(Y_l) / N)
        Y_r = Y[X[:, feat] > val]
        HYX += -self.aloga(len(Y_r) / N)
        return HYX

    # 计算用feat <= val划分数据集的信息增益率
    def info_gain_ratio(self, X, Y, feat, val):
        IG = self.info_gain(X, Y, feat, val)
        HYX = self.entropy_YX(X, Y, feat, val)
        return IG / HYX

    # 用ID3算法递归分裂节点，构造决策树
    def ID3(self, node, X, Y):
        # 判断是否已经分类完成
        if len(np.unique(Y)) == 1:
            node.feat = Y[0]
            self.T += 1
            return
        
        # 寻找最优分类特征和分类点
        best_IGR = 0 # 最优信息增益率
        best_feat = None # 最优特征编号
        best_val = None # 最优特征取值
        for feat in range(len(feat_names)):
            for val in self.feat_ranges[feat_names[feat]]:
                IGR = self.info_gain_ratio(X, Y, feat, val)
                if IGR > best_IGR:
                    best_IGR = IGR
                    best_feat = feat
                    best_val = val
        
        # 计算用best_feat <= best_val分类带来的代价函数变化
        # 由于分裂叶节点只涉及该局部，我们只需要计算分裂前后该节点的代价函数
        # 当前代价
        cur_cost = len(Y) * self.entropy(Y) + self.lbd
        # 分裂后的代价，按best_feat的取值分类统计
        # 如果best_feat为None，说明最优的信息增益率为0，
        # 再分类也无法增加信息了，因此将new_cost设置为无穷大
        if best_feat is None:
            new_cost = np.inf
        else:
            new_cost = 0
            X_feat = X[:, best_feat] 
            # 获取划分后的两部分，计算新的熵
            new_Y_l = Y[X_feat <= best_val]
            new_cost += len(new_Y_l) * self.entropy(new_Y_l)
            new_Y_r = Y[X_feat > best_val]
            new_cost += len(new_Y_r) * self.entropy(new_Y_r)
            # 分裂后会有两个叶节点
            new_cost += 2 * self.lbd

        if new_cost <= cur_cost:
            # 如果分裂后代价更小，那么执行分裂
            node.feat = best_feat
            node.split = best_val
            l_child = Node()
            l_X = X[X_feat <= best_val]
            l_Y = Y[X_feat <= best_val]
            self.ID3(l_child, l_X, l_Y)
            r_child = Node()
            r_X = X[X_feat > best_val]
            r_Y = Y[X_feat > best_val]
            self.ID3(r_child, r_X, r_Y)
            node.child = [l_child, r_child]
        else:
            # 否则将当前节点上最多的类别作为该节点的类别
            vals, cnt = np.unique(Y, return_counts=True)
            node.feat = vals[np.argmax(cnt)]
            self.T += 1

    # 预测新样本的分类
    def predict(self, x):
        node = self.root
        # 从根节点开始向下寻找，到叶节点结束
        while node.split is not None:
            # 判断x应该处于哪个子节点
            if x[node.feat] <= node.split:
                node = node.child[0]
            else:
                node = node.child[1]
        # 到达叶节点，返回类别
        return node.feat

    # 计算在样本X，标签Y上的准确率
    def accuracy(self, X, Y):
        correct = 0
        for x, y in zip(X, Y):
            pred = self.predict(x)
            if pred == y:
                correct += 1
        return correct / len(Y)

In [43]:
result={}
for lbd in np.linspace(0.01, 1, 5): 
    # 创建决策树对象
    DT = DecisionTree(train_x, train_y, feat_ranges, lbd)
    # 计算在训练集和测试集上的准确率
    train_acc = DT.accuracy(train_x, train_y)
    test_acc = DT.accuracy(test_x, test_y)
    result[lbd] = (train_acc, test_acc)
    # print(result)
    print(f'λ={lbd:.2f}, 训练集准确率={train_acc:.4f}, 测试集准确率={test_acc:.8f}')

# 输出最大的测试集准确率和对应的λ
max_test_acc = max(result.items(), key=lambda x: x[1][1])
print(max_test_acc)
print(f'最大测试集准确率={max_test_acc[1][1]:.4f}，对应的λ={max_test_acc[0]:.2f}')


# DT = DecisionTree(train_x, train_y, feat_ranges, lbd=0.005)
# print('叶节点数量：', DT.T)

# # 计算在训练集和测试集上的准确率
# print('训练集准确率：', DT.accuracy(train_x, train_y))
# print('测试集准确率：', DT.accuracy(test_x, test_y))

λ=0.01, 训练集准确率=0.9199, 测试集准确率=0.75418994
λ=0.26, 训练集准确率=0.9185, 测试集准确率=0.76536313
λ=0.51, 训练集准确率=0.9143, 测试集准确率=0.76536313
λ=0.75, 训练集准确率=0.9101, 测试集准确率=0.76536313
λ=1.00, 训练集准确率=0.8301, 测试集准确率=0.72625698
(np.float64(0.2575), (0.9185393258426966, 0.7653631284916201))
最大测试集准确率=0.7654，对应的λ=0.26


In [38]:
from sklearn import tree

# criterion表示分类依据，max_depth表示树的最大深度
# entropy生成的是C4.5分类树
c45 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)
c45.fit(train_x, train_y)
# gini生成的是CART分类树
cart = tree.DecisionTreeClassifier(criterion='gini', max_depth=5)
cart.fit(train_x, train_y)

c45_train_pred = c45.predict(train_x)
c45_test_pred = c45.predict(test_x)
cart_train_pred = cart.predict(train_x)
cart_test_pred = cart.predict(test_x)
print(f'训练集准确率：C4.5：{np.mean(c45_train_pred == train_y)}，' \
    f'CART：{np.mean(cart_train_pred == train_y)}')
print(f'测试集准确率：C4.5：{np.mean(c45_test_pred == test_y)}，' \
    f'CART：{np.mean(cart_test_pred == test_y)}')

from six import StringIO
import pydotplus

dot_data = StringIO()
tree.export_graphviz( # 导出sklearn的决策树的可视化数据
    c45,
    out_file=dot_data,
    feature_names=feat_names,
    class_names=['non-survival', 'survival'],
    filled=True, 
    rounded=True,
    impurity=False
)
# 用pydotplus生成图像
graph = pydotplus.graph_from_dot_data(
    dot_data.getvalue().replace('\n', '')) 
graph.write_png('tree.png')

训练集准确率：C4.5：0.8567415730337079，CART：0.8623595505617978
测试集准确率：C4.5：0.8324022346368715，CART：0.8379888268156425


True